This model utilizes a DCGAN architecture

In [1]:
import torch
import music21
import numpy as np
import torch.nn as nn 



music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
class Generator(nn.Module):
    
    def __init__(self, latent_dim, output_dim, sequence_length):
        super(Generator, self).__init__()
        self.sequence_length = sequence_length
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_dim * sequence_length),
            nn.Tanh()
        )

    def forward(self, z):
        output = self.model(z)
        output = output.view(-1, self.sequence_length, output_dim)
        return output
    
        


In [3]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, sequence_length):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim * sequence_length, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # flatten the input
        output = self.model(x)
        return output


In [4]:
# Loss function
adversarial_loss = nn.BCELoss()

# Initialize generator and discriminator
latent_dim = 100  # size of random noise
sequence_length = 128  # length of each music sequence
output_dim = inputs.shape[2]  # dimension of the music sequence (vocabulary size)

generator = Generator(latent_dim, output_dim, sequence_length).to(device)
discriminator = Discriminator(output_dim, sequence_length).to(device)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


NameError: name 'inputs' is not defined

In [ ]:
import numpy as np
import tqdm

epochs = 10000
batch_size = 32

for epoch in range(epochs):
    for i, (real_inputs, _, _) in enumerate(dataloader):

        # Adversarial ground truths
        real_labels = torch.ones((real_inputs.size(0), 1), device=device)
        fake_labels = torch.zeros((real_inputs.size(0), 1), device=device)

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Train on real inputs
        real_inputs = real_inputs.to(device)
        real_validity = discriminator(real_inputs)
        real_loss = adversarial_loss(real_validity, real_labels)

        # Train on fake inputs
        z = torch.randn(real_inputs.size(0), latent_dim, device=device)
        fake_inputs = generator(z)
        fake_validity = discriminator(fake_inputs.detach())
        fake_loss = adversarial_loss(fake_validity, fake_labels)

        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Generator tries to fool the discriminator
        fake_validity = discriminator(fake_inputs)
        g_loss = adversarial_loss(fake_validity, real_labels)

        g_loss.backward()
        optimizer_G.step()

    print(f"[Epoch {epoch}/{epochs}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
